In [57]:
import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
import scipy
import math
import os
import sys
print(f'python version: {sys.version}')
print(f'numpy version: {np.__version__}')
print(f'pandas version: {pd.__version__}')
print(f'nibabel version: {nib.__version__}')


NEAR_ZERO_THRESHOLD = 1e-6
def int2seg(intensity):
    if intensity <= 7:
        return f"C{intensity}"
    elif intensity >7 and intensity <= 19:
        return f"T{intensity-7}"
    else:
        return f"L{intensity-19}"
    
df = pd.read_csv("mendez_df.csv")
df.head()

python version: 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]
numpy version: 1.16.5
pandas version: 0.25.1
nibabel version: 3.1.1


,Segment,MEAN,STD,Mesure
0,C2,11.3,1.5,DREZ length
1,C2,12.6,0.9,Seg.L at dorsal CE
2,C2,13.1,1.0,Seg.L at BE
3,C2,24.4,0.8,Inf.Art facet to CR distance
4,C2,15.5,4.2,IF to RR distance


In [58]:
sct_dir = 'c:/Users/sb199/spinalcordtoolbox/data/PAM50/template/'
print(sct_dir)

c:/Users/sb199/spinalcordtoolbox/data/PAM50/template/


In [59]:
# Import foramen data
#foramen_level = nib.load('PAM50_intervertebral_foramen_sandrine.nii.gz') # TODO add path
foramen_level = nib.load('test_foramen.nii.gz')

x_f,y_f,z_f = np.where(foramen_level.get_fdata()>NEAR_ZERO_THRESHOLD)
print(x_f, z_f)

[37 40 41 41 42 42 42 42] [932 731 691 898 765 797 830 865]


In [60]:
# Import Centerline data
center = nib.load(os.path.join(sct_dir, 'PAM50_centerline.nii.gz'))
x_c, y_c, z_c = np.where(center.get_fdata() > NEAR_ZERO_THRESHOLD)
coord_ctl = np.where(center.get_fdata() > NEAR_ZERO_THRESHOLD)


# Import Cord data
PATH = os.path.join(sct_dir, 'PAM50_cord.nii.gz')
cord = nib.load(PATH)
p50_mask = cord.get_fdata()
x_cord, y_cord, z_cord = np.where(p50_mask>0)

In [61]:
# Take smallest distance bewtenn SC centerline (shifted by average diameter at dorsal rootlet entry and intervertebral foramen) (Sandrine)

len_level = {}
z_ref =  np.array(range(min(z_c), max(z_c) + 1))
z_c = z_ref
for i in range(len(x_f)):
    # Get coordinates of intervertebral foramen
    x = x_f[i]
    y = y_f[i]
    z = z_f[i]
    # Get level
    level = int(foramen_level.get_fdata()[x,y,z])
    print('Level', level)
    # Get correspondance of level number (2 --> C2, etc.)
    seg = int2seg(level)
    # Get mendez measures for the corresponding level
    seg_df = df[df.Segment == seg]
    RR = seg_df[seg_df.Mesure == "IF to RR distance"]
    CR = seg_df[seg_df.Mesure == "IF to CR distance"]
    W = seg_df.loc[seg_df.Mesure == "Dorsal width"].iloc[0]
    w_mean = W.MEAN # get mean value of dorsal width
    rr_mean = RR.MEAN.to_list()[0] # Get mean distance from rostral rootlet to interverteral foramen
    cr_mean = CR.MEAN.to_list()[0] # Get mean distance from caudal rootlet to interverteral foramen
    # Compute the 3D euclidean distance between intervertebral foramen and the entire centerline ( in mm)
    distance_foramen_ctl = np.sqrt((x_c - w_mean*2 - x)**2 + (y_c - y)**2  + (z_c - z)**2)*0.5  # 0.5 --> pix dim of PAM50 TODO: remove hardcode

    # Get the closest distance to the rostral rootlet compared to the Mendez value
    rostral_diff = np.array([np.abs(i - rr_mean) for i in distance_foramen_ctl])
    # Only use slices higher than the foramen
    rostral = np.argmin(rostral_diff[-len(z_c[z_c>z])-1::])
    z_ref_r = z_ref[-len(z_c[z_c>z])-1::]
    # Get the slice number (adjusted since the centerline starts at slice 55 not 0)
    r_z = z_ref_r[rostral]

    # Get the closest distance to the caudal rootlet compared to the Mendez value
    caudal_diff = np.array([np.abs(i - cr_mean) for i in distance_foramen_ctl])
    # Only use slices higher than the foramen
    caudal = np.argmin(caudal_diff[-len(z_c[z_c>z])-1::])
    # Get the slice number (adjusted since the centerline starts at slice 55 not 0)
    c_z = z_ref_r[caudal]

    print('Rostral', rr_mean, distance_foramen_ctl[-len(z_c[z_c>z])-1::][rostral], r_z, z)
    print('Caudal', cr_mean, distance_foramen_ctl[-len(z_c[z_c>z])-1::][caudal], c_z, z)
    # Set slices
    len_level[level] = (r_z, c_z)


Level 2
Rostral 15.5 15.606229525417087 958 932
Caudal 10.6 10.51448524655392 944 932
Level 8
Rostral 28.1 28.13201023745015 782 731
Caudal 17.0 16.886681142249355 755 731
Level 9
Rostral 33.4 33.395162823379074 754 691
Caudal 22.0 21.999702270712664 729 691
Level 3
Rostral 16.299999999999994 16.33500535659539 921 898
Caudal 10.8 11.600965477062674 898 898
Level 7
Rostral 20.1 20.01125683209328 798 765
Caudal 11.8 11.851177156721606 772 765
Level 6
Rostral 17.2 17.044952331995535 823 797
Caudal 10.3 11.02408272828175 797 797
Level 5
Rostral 15.1 15.164831024446004 851 830
Caudal 9.3 10.941759456321456 830 830
Level 4
Rostral 18.1 18.165134186127005 894 865
Caudal 10.1 10.941759456321456 865 865


In [62]:
# # Test triangle (Théo)
# len_level = {}
# for i in range(len(x_f)):
#     x = x_f[i]
#     y = y_f[i]
#     z = z_f[i]
#     intensity = int(foramen_level.get_fdata()[x,y,z])
#     print('Level', intensity)
#     seg = int2seg(intensity)
#     seg_df = df[df.Segment == seg]
#     RR = seg_df[seg_df.Mesure == "IF to RR distance"]
#     CR = seg_df[seg_df.Mesure == "IF to CR distance"]

#     center_val = np.where(z_c == z)

#     try :
#         x_y_c = [x_c[center_val][0], y_c[center_val][0]]
#     except:
#         x_y_c = [70, 70]
#     dist_c = np.sqrt(((x-x_y_c[0])**2 + (y-x_y_c[1])**2))

#    # dist_c = math.dist([x,y], x_y_c)
#     # Because we take the centerline for some case, no estimation based on a triangle can be made, so let the estimation at 0.
#     try:
#         len_RR = math.sqrt((RR.MEAN*2)**2 - dist_c**2)
#     except :
#         len_RR = 0
#     try:
#         len_CR = math.sqrt((CR.MEAN*2)**2 - dist_c**2)
#     except:
#         len_CR = 0
#     len_level[intensity] = (z+len_RR, z+len_CR)
#     print('R:', z+len_RR, z)
#     print('C:', z+len_CR, z)

In [63]:
cord_mask = np.copy(p50_mask)
for lvl in len_level:
    level_r = int(len_level[lvl][0])
    level_c = int(len_level[lvl][1])
    cord_mask[:, :, level_r][cord_mask[:, :, level_r]>0] = lvl
    cord_mask[:, :, level_c][cord_mask[:, :, level_c]>0] = lvl
cord_mask[:, :, :][cord_mask[:, :, :]==1] = 0
test_img = nib.Nifti1Image(cord_mask, header=cord.header, affine=cord.affine)
nib.save(test_img, f'test_sb_v3.nii.gz')  # TO CHANGE path output